code to get group users info

In [ ]:
from snowflake.snowpark import Session
import json as js
import requests
from snowflake.snowpark.functions import col
import pandas as pd
from requests.exceptions import RequestException, HTTPError
import time

# Get active Snowflake session
session = get_active_session()
pd.options.mode.chained_assignment = None
# Fetch client ID and secret from Snowflake secure function
try:
    result = session.sql("SELECT TEST_SCHEMA.GET_SECRET_VAL()").collect()
    raw_result = result[0][0]  # Extract the first value of the first row
    parsed_result = raw_result.strip('()').split(',')  # Remove parentheses and split by comma

    # Extract and clean values
    cl_id_raw = parsed_result[0].strip().replace("'", '')  # Clean raw client ID string
    client_secret = parsed_result[1].strip().replace("'", '')  # Clean raw client secret string
except Exception as e:
    raise RuntimeError(f"Failed to fetch or parse client credentials: {e}")

# Microsoft OAuth2 Token URL
url = "https://login.microsoftonline.com/488d0bae-755e-4d30-9fb8-e0c5495654c4/oauth2/v2.0/token"

# Prepare payload for token request
payload = f"grant_type=client_credentials&scope=https://graph.microsoft.com/.default&client_id={cl_id_raw}&client_secret={client_secret}"
headers = {"Content-Type": "application/x-www-form-urlencoded"}

# Get OAuth2 token
try:
    response = requests.post(url, data=payload, headers=headers)
    response.raise_for_status()  # Raise HTTPError for bad responses
    token_js = js.loads(response.text)
    token = token_js["access_token"]
except (RequestException, HTTPError) as e:
    raise RuntimeError(f"Failed to fetch access token: {e}")
except KeyError:
    raise RuntimeError("Access token not found in the response")

def load_groups(url_vl, token, df=None):
    """
    Recursively loads all groups from a given API URL and appends them to a DataFrame.

    Args:
        url_vl (str): The URL to fetch group data.
        token (str): Bearer token for authentication.
        df (pd.DataFrame, optional): DataFrame to append data to. Defaults to None.

    Returns:
        pd.DataFrame: DataFrame containing all loaded groups.
    """
    if df is None:
        df = pd.DataFrame()

    if not token:
        raise ValueError("Token must be provided")

    headers = {
        'Authorization': f"Bearer {token}"
    }

    try:
        response = requests.get(url_vl, headers=headers)
        response.raise_for_status()
        response_js = js.loads(response.text)
        print(response_js.get('value', []))

        # Extract and append data to the DataFrame
        current_df = pd.DataFrame(response_js.get('value', []))
        df = pd.concat([df, current_df], ignore_index=True)

        # Recursively load next pages if available
        next_link = response_js.get("@odata.nextLink")
        if next_link:
            return load_groups(next_link, token, df)

    except KeyError as e:
        print(f"Key error encountered: {e}")

    return df


df = load_groups(url_vl=f"https://graph.microsoft.com/v1.0/groups/77f267e1-b057-46b5-9ace-e7c4ba699ed0/members", token=token)
#df.to_csv("USERS.csv")
print('Insert Starts')
    # Write data to Snowflake
session.write_pandas(df, schema='TEST_SCHEMA', table_name="RLS_USERS", overwrite=True, auto_create_table=True    )
#print(df)

In [ ]:
select * from TEST_DB.TEST_SCHEMA.RLS_USERS 

In [ ]:
CREATE TABLE TEST_DB.TEST_SCHEMA.SALES (REGION STRING, SALES_AMT INT)

In [ ]:
INSERT INTO TEST_DB.TEST_SCHEMA.SALES VALUES('APAC',90000) ;
INSERT INTO TEST_DB.TEST_SCHEMA.SALES VALUES('AMER',120000);
INSERT INTO TEST_DB.TEST_SCHEMA.SALES VALUES('EMEA',100000);

In [ ]:
SELECT * FROM TEST_DB.TEST_SCHEMA.SALES;

code to get location info alone

In [ ]:
from snowflake.snowpark import Session
import json as js
import requests
from snowflake.snowpark.functions import col
import pandas as pd
from requests.exceptions import RequestException, HTTPError
import time

# Get active Snowflake session
session = get_active_session()
pd.options.mode.chained_assignment = None
# Fetch client ID and secret from Snowflake secure function
try:
    result = session.sql("SELECT TEST_SCHEMA.GET_SECRET_VAL()").collect()
    raw_result = result[0][0]  # Extract the first value of the first row
    parsed_result = raw_result.strip('()').split(',')  # Remove parentheses and split by comma

    # Extract and clean values
    cl_id_raw = parsed_result[0].strip().replace("'", '')  # Clean raw client ID string
    client_secret = parsed_result[1].strip().replace("'", '')  # Clean raw client secret string
except Exception as e:
    raise RuntimeError(f"Failed to fetch or parse client credentials: {e}")

# Microsoft OAuth2 Token URL
url = "https://login.microsoftonline.com/488d0bae-755e-4d30-9fb8-e0c5495654c4/oauth2/v2.0/token"

# Prepare payload for token request
payload = f"grant_type=client_credentials&scope=https://graph.microsoft.com/.default&client_id={cl_id_raw}&client_secret={client_secret}"
headers = {"Content-Type": "application/x-www-form-urlencoded"}

# Get OAuth2 token
try:
    response = requests.post(url, data=payload, headers=headers)
    response.raise_for_status()  # Raise HTTPError for bad responses
    token_js = js.loads(response.text)
    token = token_js["access_token"]
except (RequestException, HTTPError) as e:
    raise RuntimeError(f"Failed to fetch access token: {e}")
except KeyError:
    raise RuntimeError("Access token not found in the response")

def load_groups(url_vl, token, df=None):
    """
    Recursively loads all groups from a given API URL and appends them to a DataFrame.

    Args:
        url_vl (str): The URL to fetch group data.
        token (str): Bearer token for authentication.
        df (pd.DataFrame, optional): DataFrame to append data to. Defaults to None.

    Returns:
        pd.DataFrame: DataFrame containing all loaded groups.
    """
    if df is None:
        df = pd.DataFrame()

    if not token:
        raise ValueError("Token must be provided")

    headers = {
        'Authorization': f"Bearer {token}"
    }

    try:
        response = requests.get(url_vl, headers=headers)
        response.raise_for_status()
        response_js = js.loads(response.text)
        print(response_js.get('value', []))

        # Extract and append data to the DataFrame
        current_df = pd.DataFrame(response_js.get('value', []))
        df = pd.concat([df, current_df], ignore_index=True)

        # Recursively load next pages if available
        next_link = response_js.get("@odata.nextLink")
        if next_link:
            return load_groups(next_link, token, df)

    except KeyError as e:
        print(f"Key error encountered: {e}")

    return df


df = load_groups(url_vl=f"https://graph.microsoft.com/v1.0/users?$select=userPrincipalName,usageLocation", token=token)
#df.to_csv("USERS.csv")
print('Insert Starts')
    # Write data to Snowflake
session.write_pandas(df, schema='TEST_SCHEMA', table_name="RLS_USERS", overwrite=True, auto_create_table=True    )
#print(df)

Latest code with group users and location details

In [ ]:
from snowflake.snowpark import Session
import json as js
import requests
import pandas as pd
from requests.exceptions import RequestException, HTTPError

# Get active Snowflake session
session = get_active_session()
pd.options.mode.chained_assignment = None

# Fetch client ID and secret from Snowflake secure function
try:
    result = session.sql("SELECT TEST_SCHEMA.GET_SECRET_VAL()").collect()
    raw_result = result[0][0]  # Extract the first value of the first row
    parsed_result = raw_result.strip('()').split(',')  # Remove parentheses and split by comma

    # Extract and clean values
    cl_id_raw = parsed_result[0].strip().replace("'", '')  # Clean raw client ID string
    client_secret = parsed_result[1].strip().replace("'", '')  # Clean raw client secret string
except Exception as e:
    raise RuntimeError(f"Failed to fetch or parse client credentials: {e}")

# Microsoft OAuth2 Token URL
url = "https://login.microsoftonline.com/488d0bae-755e-4d30-9fb8-e0c5495654c4/oauth2/v2.0/token"

# Prepare payload for token request
payload = f"grant_type=client_credentials&scope=https://graph.microsoft.com/.default&client_id={cl_id_raw}&client_secret={client_secret}"
headers = {"Content-Type": "application/x-www-form-urlencoded"}

# Get OAuth2 token
try:
    response = requests.post(url, data=payload, headers=headers)
    response.raise_for_status()  # Raise HTTPError for bad responses
    token_js = js.loads(response.text)
    token = token_js["access_token"]
except (RequestException, HTTPError) as e:
    raise RuntimeError(f"Failed to fetch access token: {e}")
except KeyError:
    raise RuntimeError("Access token not found in the response")

def load_groups(url_vl, token, df=None):
    """
    Recursively loads all groups from a given API URL and appends them to a DataFrame.

    Args:
        url_vl (str): The URL to fetch group data.
        token (str): Bearer token for authentication.
        df (pd.DataFrame, optional): DataFrame to append data to. Defaults to None.

    Returns:
        pd.DataFrame: DataFrame containing all loaded groups.
    """
    if df is None:
        df = pd.DataFrame()

    if not token:
        raise ValueError("Token must be provided")

    headers = {
        'Authorization': f"Bearer {token}"
    }

    try:
        response = requests.get(url_vl, headers=headers)
        response.raise_for_status()
        response_js = js.loads(response.text)
        print(response_js.get('value', []))

        # Extract and append data to the DataFrame
        current_df = pd.DataFrame(response_js.get('value', []))
        df = pd.concat([df, current_df], ignore_index=True)

        # Recursively load next pages if available
        next_link = response_js.get("@odata.nextLink")
        if next_link:
            return load_groups(next_link, token, df)

    except KeyError as e:
        print(f"Key error encountered: {e}")

    return df


# Fetching the members of a specific group
df = load_groups(url_vl=f"https://graph.microsoft.com/v1.0/groups/77f267e1-b057-46b5-9ace-e7c4ba699ed0/members", token=token)

# Fetch the userPrincipalName and usageLocation for each user
users_df = load_groups(url_vl="https://graph.microsoft.com/v1.0/users?$select=userPrincipalName,usageLocation", token=token)

# Merge the userPrincipalName and usageLocation data with the existing group member data
# Here we assume userPrincipalName is a common column in both DataFrames for merging
df_merged = pd.merge(df, users_df[['userPrincipalName', 'usageLocation']], on='userPrincipalName', how='left')

# Insert the updated merged data back to Snowflake
print('Insert Starts')
session.write_pandas(df_merged, schema='TEST_SCHEMA', table_name="RLS_USERS", overwrite=True, auto_create_table=True)
print('Insert Completed')
